In [4]:
import datetime
import time
import requests

# Archive.org crawling

In [20]:
#https://stackoverflow.com/questions/28154066/how-to-convert-datetime-to-integer-in-python
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [21]:
#https://stackoverflow.com/questions/993358/creating-a-range-of-dates-in-python
start = datetime.datetime(2017, 12, 30)
dates = [start - datetime.timedelta(days=x) for x in range(0, 1500)]

In [22]:
intdates = [to_integer(date) for date in dates]

In [ ]:
not_testing = []
for i in intdates:
    r = requests.get('http://archive.org/wayback/available?url=nytimes.com&timestamp={}'.format(str(i)))
    arc = r.json()['archived_snapshots']['closest']['url']
    r2 = requests.get(arc)
    if 'optimizely.com' not in r2.text:
        print(arc)
        not_testing.append(arc)
    else:
        print(i)
        time.sleep(2)

In [ ]:
#http://web.archive.org/web/20150805230843/http://www.nytimes.com:80/

# NYT headlines

Format: http://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={your-api-key}

In [29]:
key = %env KEY

def query_range(startmo, startyr, endmo, endyr):
    startmos = [[i, startyr] for i in range(startmo, 13)]
    middle = [[j, i] for i in range(startyr, endyr+1) if i != startyr and i != endyr for j in range(1, 13)]
    endmos = [[i, endyr] for i in range(1, endmo+1)]
    allmos = [i for sub in [startmos, middle, endmos] for i in sub]
    return allmos

def cleaning(entry):
    key_list = ['web_url', 'headline', 'keywords', 'pub_date', 'document_type', 'new_desk', 'word_count']
    filtered = {new_key: entry[new_key] for new_key in key_list}
    try:
        filtered['print_headline'] = filtered['headline']['print_headline']
    except Exception:
        pass
    filtered['headline'] = filtered['headline']['main']
    for idx, i in enumerate(filtered['keywords']):
        filtered['keyword{}'.format(idx)] = i
    filtered.pop('keywords', None)
    return filtered

In [ ]:
morange = query_range(5, 2012, 9, 2018)

for i in morange:
    h = requests.get('http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(i[1], i[0], key))
    items = h.json()['response']['docs']
    cleaned = [cleaning(i) for i in items]

In [37]:
h = requests.get('http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(2015, 1, key))
items = h.json()['response']['docs']
items[1].keys()

dict_keys(['web_url', 'snippet', 'lead_paragraph', 'abstract', 'print_page', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 'slideshow_credits'])

In [35]:
items[10].keys()

dict_keys(['web_url', 'snippet', 'print_page', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'new_desk', 'byline', 'type_of_material', '_id', 'word_count', 'score', 'uri'])